In [4]:
import sys

from cobra.io import read_sbml_model

sys.path.insert(0, r"C:\Users\Bisbii\PythonProjects\ExpAlgae\src")
from ExpAlgae.model.COBRAmodel import *
from cobra.flux_analysis import pfba, find_blocked_reactions, flux_variability_analysis as fva

In [49]:
data_directory = "../data"
model = MyModel(join(data_directory, "models/model_with_trials.xml"), "e_Biomass__cytop")
model.add_medium(join(data_directory, "media.xlsx"), "base_medium")
model.setup_condition("4")

Loading

Reactions: 4582
Metabolites: 3655
Genes: 1684
Model loaded


In [46]:
model.metabolites.C00065__mito.summary(fva=1.0)

Percent,Flux,Range,Reaction,Definition
27.45%,0.08217,[-1000; 1000],R00588__mito,C00048__mito + C00065__mito <=> C00037__mito + C00168__mito
72.55%,0.2172,[-25.69; 10.37],R00945__mito,C00001__mito + C00037__mito + C00080__mito + C00143__mito <=> C00065__mito + C00101__mito
Percent,Flux,Range,Reaction,Definition
100.00%,-0.2994,[-1000; 1000],T_L_SERtm__mitmem,C00065__mito + C00080__mito <=> C00065__cytop + C00080__cytop


In [48]:
with model:
    # model.create_sink("C00148__mito")
    print(fva(model, ['R02236__cytop'], fraction_of_optimum=0.0))

                minimum  maximum
R02236__cytop -0.004173      0.0


In [45]:
model.test_reaction("R02236__cytop")

,Flux
Folate,0.00000
NADPH,0.20339
H+,0.20339


In [3]:
model.metabolites.C00025__mito.summary(fva = 1.0)

Percent,Flux,Range,Reaction,Definition
38.95%,0.3992,[-53.89; 42.16],R00243__mito,C00001__mito + C00003__mito + C00025__mito <=> C00004__mito + C00014__mito + C00026__mito + C00080__mito
0.00%,0,[0; 25.65],R00245__mito,C00001__mito + C00003__mito + C01165__mito --> C00004__mito + C00025__mito + 2.0 C00080__mito
0.00%,0,[0; 25.65],R00667__mito,C00026__mito + C00077__mito <=> C00025__mito + C01165__mito
0.00%,0,[0; 8.321],R00895__mito,C00026__mito + C00097__mito <=> C00025__mito + C00957__mito
61.05%,0.6257,[-31.16; 40],R01090__mito,C00026__mito + C00123__mito <=> C00025__mito + C00233__mito
0.00%,0,[0; 3.052],R01648__mito,C00026__mito + C00334__mito <=> C00025__mito + C00232__mito
0.00%,0,[0; 41.46],R04242__mito,2.0 C00001__mito + C04144__mito --> 2.0 C00025__mito + C00101__mito
0.00%,0,[0; 1000],R10991__mito,C00026__mito + C02356__mito <=> C00025__mito + C00109__mito
Percent,Flux,Range,Reaction,Definition
5.39%,-0.05523,[-41.52; 43.45],R00248__mito,C00005__mito + C00014__mito + C00026__mito + C00080__mito <=> C00001__mito + C00006__mito + C00025__mito


In [37]:
model.summary()

Runnning pFBA


Metabolite,Reaction,Flux,C-Number,C-Flux
C00001__extr,EX_C00001__dra,7.206,0,0.00%
C00009__extr,EX_C00009__dra,0.03358,0,0.00%
C00011__extr,EX_C00011__dra,8.77,1,99.76%
C00059__extr,EX_C00059__dra,0.02233,0,0.00%
C00080__extr,EX_C00080__dra,0.05665,0,0.00%
C00120__extr,EX_C00120__dra,0.0008064,10,0.09%
C00205__extr,EX_C00205__dra,621.8,0,0.00%
C00244__extr,EX_C00244__dra,0.5524,0,0.00%
C00305__extr,EX_C00305__dra,0.0001709,0,0.00%
C00378__extr,EX_C00378__dra,0.0004632,12,0.06%


In [39]:
#create copy reactions
with model:
    carbohydrate_copy = model.reactions.e_Carbohydrate__cytop.copy()
    carbohydrate_copy.id = 'e_Carbohydrate_no_starch__cytop'
    biomass_copy = model.reactions.e_Biomass__cytop.copy()
    biomass_copy.id = 'e_ActiveBiomass__cytop'
    model.add_reactions([carbohydrate_copy, biomass_copy])
    model.objective = 'e_ActiveBiomass__cytop'
    model.reactions.e_Carbohydrate__cytop.bounds = (0, 0)
    #remove starch from e_carbohydrate
    carbohydrate_copy.add_metabolites({model.metabolites.C00369__chlo: -carbohydrate_copy.metabolites[model.metabolites.C00369__chlo]})

    #adjust other carbohydrates
    carbohydrate_copy.add_metabolites({model.metabolites.C00052__cytop:  -3.35762291 - carbohydrate_copy.metabolites[model.metabolites.C00052__cytop]})
    carbohydrate_copy.add_metabolites({model.metabolites.C00096__cytop:  -1.58181346 - carbohydrate_copy.metabolites[model.metabolites.C00096__cytop]})
    carbohydrate_copy.add_metabolites({model.metabolites.C00935__cytop:  -2.52493299 - carbohydrate_copy.metabolites[model.metabolites.C00935__cytop]})

    carbohydrate_copy.add_metabolites({model.metabolites.C00015__cytop:  5.8825559 - carbohydrate_copy.metabolites[model.metabolites.C00015__cytop]})
    carbohydrate_copy.add_metabolites({model.metabolites.C00035__cytop:  1.58181346 - carbohydrate_copy.metabolites[model.metabolites.C00035__cytop]})
    carbohydrate_copy.add_metabolites({model.metabolites.C00001__cytop:  7.46436936 - carbohydrate_copy.metabolites[model.metabolites.C00001__cytop]})

    print(carbohydrate_copy.metabolites)

    biomass_copy.add_metabolites({model.metabolites.e_Carbohydrate__cytop:  -0.203306526 - biomass_copy.metabolites[model.metabolites.e_Carbohydrate__cytop]})
    print(biomass_copy.metabolites)
    print(model.optimize().objective_value*0.7083)

    # total = abs(sum([biomass_copy.metabolites[met] for met in biomass_copy.reactants if biomass_copy.metabolites[met] > -10]))
    # for  met in biomass_copy.reactants:
    #     if biomass_copy.metabolites[met] > -10:
    #         print(met.id)
    #         new_st = biomass_copy.metabolites[met]/total
    #         print(new_st)
    #         print(biomass_copy.metabolites[met])
    #         biomass_copy.add_metabolites({model.metabolites.get_by_id(met.id):  new_st - biomass_copy.metabolites[met]})
    #         print(biomass_copy.metabolites[met])
    # print(sum([biomass_copy.metabolites[met] for met in biomass_copy.reactants if biomass_copy.metabolites[met] > -10]))
    # print(model.optimize().objective_value)
    model.write(os.path.join(data_directory, "models/model_dfba.xml"))

{<Metabolite C00096__cytop at 0x2d492e90220>: -1.58181346, <Metabolite C00052__cytop at 0x2d492a55df0>: -3.35762291, <Metabolite C00935__cytop at 0x2d49328c790>: -2.52493299, <Metabolite C00015__cytop at 0x2d492ebe310>: 5.8825559, <Metabolite e_Carbohydrate__cytop at 0x2d492a55970>: 1.0, <Metabolite C00035__cytop at 0x2d492a55a30>: 1.58181346, <Metabolite C00001__cytop at 0x2d4932ba460>: 7.46436936}
{<Metabolite e_Pigment__chlo at 0x2d492b35610>: -0.02, <Metabolite e_Acid__cytop at 0x2d4932a43a0>: -0.15678, <Metabolite e_RNA__cytop at 0x2d492a55fd0>: -0.02, <Metabolite e_Carbohydrate__cytop at 0x2d492a55970>: -0.203306526, <Metabolite e_Lipid__cytop at 0x2d492a826d0>: -0.111, <Metabolite C00002__cytop at 0x2d492e90b50>: -39.24, <Metabolite C00001__cytop at 0x2d4932ba460>: -39.24, <Metabolite e_Sterol__cytop at 0x2d4932ba520>: -0.0014, <Metabolite e_DNA__cytop at 0x2d4932a4ca0>: -0.01, <Metabolite e_Cofactor__cytop at 0x2d492a98760>: -0.02312, <Metabolite e_Protein__cytop at 0x2d492a6b5

In [23]:
data_directory = "../data"
model = MyModel(join(data_directory, "model_dfba.xml"), "e_Biomass_trial4__cytop")
model.add_medium(join(data_directory, "media.xlsx"), "base_medium")
model.setup_condition("4")

Loading

Reactions: 4652
Metabolites: 3859
Genes: 3184
Model loaded


In [24]:
pfba_sol = pfba(model)
model.summary(pfba_sol)

Runnning pFBA


Metabolite,Reaction,Flux,C-Number,C-Flux
C00001__extr,EX_C00001__dra,6.709,0,0.00%
C00009__extr,EX_C00009__dra,0.02764,0,0.00%
C00011__extr,EX_C00011__dra,8.77,1,100.00%
C00059__extr,EX_C00059__dra,0.01377,0,0.00%
C00080__extr,EX_C00080__dra,0.04141,0,0.00%
C00205__extr,EX_C00205__dra,609.8,0,0.00%
C00244__extr,EX_C00244__dra,0.4444,0,0.00%
C00305__extr,EX_C00305__dra,2.304E-05,0,0.00%
C14818__extr,EX_C14818__dra,2.073E-07,0,0.00%
Metabolite,Reaction,Flux,C-Number,C-Flux


In [25]:
with model:
    model.reactions.DM_C00369__chlo.bounds = (0.00147, 1000)
    pfba_sol = pfba(model)
    # print(model.metabolites.C00369__chlo.summary(pfba_sol))
    print(model.summary(pfba_sol))

Runnning pFBA
Objective
1.0 e_Biomass_trial4__cytop = 0.14560173637026494

Uptake
------
  Metabolite       Reaction      Flux  C-Number  C-Flux
C00001__extr EX_C00001__dra     6.892         0   0.00%
C00009__extr EX_C00009__dra    0.0193         0   0.00%
C00011__extr EX_C00011__dra      8.77         1 100.00%
C00059__extr EX_C00059__dra  0.009616         0   0.00%
C00080__extr EX_C00080__dra   0.02891         0   0.00%
C00205__extr EX_C00205__dra     576.3         0   0.00%
C00244__extr EX_C00244__dra    0.3103         0   0.00%
C00305__extr EX_C00305__dra 1.609E-05         0   0.00%
C14818__extr EX_C14818__dra 1.448E-07         0   0.00%

Secretion
---------
      Metabolite            Reaction     Flux  C-Number  C-Flux
    C00369__chlo     DM_C00369__chlo -0.00147      1800 100.00%
photon298__cytop DM_photon298__cytop  -0.1281         0   0.00%
 photon450__chlo  DM_photon450__chlo   -15.76         0   0.00%
 photon490__chlo  DM_photon490__chlo   -87.52         0   0.00%
 photon646

In [87]:
sum([model.reactions.e_Biomass_trial4__cytop.metabolites[reactant] for reactant in model.reactions.e_Biomass_trial4__cytop.metabolites if model.reactions.e_Biomass_trial4__cytop.metabolites[reactant] < 0 and model.reactions.e_Biomass_trial4__cytop.metabolites[reactant]>-1])

-0.6945877189999999

In [31]:
original_sol = model.optimize().objective_value

In [61]:
model.summary()

Runnning pFBA


Metabolite,Reaction,Flux,C-Number,C-Flux
C00001__extr,EX_C00001__dra,6.489,0,0.00%
C00009__extr,EX_C00009__dra,0.06328,0,0.00%
C00011__extr,EX_C00011__dra,8.77,1,100.00%
C00059__extr,EX_C00059__dra,0.03695,0,0.00%
C00080__extr,EX_C00080__dra,0.1002,0,0.00%
C00120__extr,EX_C00120__dra,1.012E-07,10,0.00%
C00205__extr,EX_C00205__dra,697.7,0,0.00%
C00244__extr,EX_C00244__dra,1.218,0,0.00%
C00305__extr,EX_C00305__dra,4.63E-05,0,0.00%
C14818__extr,EX_C14818__dra,4.901E-07,0,0.00%


In [35]:
model.metabolites.C00116__cytop.formula_weight

92.09382

In [32]:
fva(model, ["T_SUCBZth__chlomem"])

,minimum,maximum
T_SUCBZth__chlomem,-2.295578e-08,0.0


In [99]:
with model:
    # model.biomass_reaction = model.bio_reaction.id
    # model.reactions.e_Carbohydrate__cytop.add_metabolites({model.metabolites.C00369__chlo: -model.reactions.e_Carbohydrate__cytop.metabolites[model.metabolites.C00369__chlo]})
    # model.reactions.e_Biomass_trial4__cytop
    # model.reactions.e_Acid__cytop.add_metabolites({model.metabolites.C00116__cytop: -model.reactions.e_Acid__cytop.metabolites[model.metabolites.C00116__cytop]})
    # model.reactions.DM_C00369__chlo.bounds = (0.305381773/model.metabolites.C00369__chlo.formula_weight*1000*original_sol, 1000)
    # model.reactions.DM_C00369__chlo.bounds = (0.00147, 1000)
    # [setattr(x, 'objective_coefficient', 0) for x in model.reactions if x.objective_coefficient != 0]
    # model.reactions.EX_e_Biomass__dra.objective_coefficient = 0.7
    # model.reactions.DM_C00369__chlo.objective_coefficient = 0.3
    # model.objective = "e_Biomass__cytop"
    sol = pfba(model)
    print(model.summary(solution=sol))
    # print(model.metabolites.C00369__chlo.summary(sol))

Objective
1.0 e_Biomass_trial4__cytop = 0.3005351891439121

Uptake
------
  Metabolite       Reaction      Flux  C-Number  C-Flux
C00001__extr EX_C00001__dra      7.22         0   0.00%
C00009__extr EX_C00009__dra    0.0409         0   0.00%
C00011__extr EX_C00011__dra      8.77         1 100.00%
C00059__extr EX_C00059__dra   0.02354         0   0.00%
C00080__extr EX_C00080__dra   0.06444         0   0.00%
C00120__extr EX_C00120__dra 1.127E-07        10   0.00%
C00205__extr EX_C00205__dra     664.6         0   0.00%
C00244__extr EX_C00244__dra    0.7615         0   0.00%
C00305__extr EX_C00305__dra  3.41E-05         0   0.00%
C14818__extr EX_C14818__dra 3.061E-07         0   0.00%

Secretion
---------
      Metabolite            Reaction    Flux  C-Number C-Flux
photon298__cytop DM_photon298__cytop -0.1477         0  0.00%
 photon450__chlo  DM_photon450__chlo  -18.18         0  0.00%
 photon490__chlo  DM_photon490__chlo  -100.9         0  0.00%
 photon646__chlo  DM_photon646__chlo  -17

In [38]:
total = sum([reactant for reactant in model.reactions.e_Biomass__cytop.metabolites if model.reactions.e_Biomass__cytop.metabolites[reactant] < 0 and model.reactions.e_Biomass__cytop.metabolites[reactant]>-1])
for reactant in model.reactions.e_Biomass__cytop.reactants:
    st =

{<Metabolite e_Pigment__chlo at 0x1f57f3bf850>: -0.02055,
 <Metabolite e_Acid__cytop at 0x1f57f258f10>: -0.15624,
 <Metabolite e_RNA__cytop at 0x1f57f2dfee0>: -0.02055,
 <Metabolite e_Carbohydrate__cytop at 0x1f57f2df850>: -0.50869,
 <Metabolite e_Lipid__cytop at 0x1f57f30e160>: -0.11407,
 <Metabolite C00002__cytop at 0x1f57f29a880>: -39.24,
 <Metabolite C00001__cytop at 0x1f57f26ec40>: -39.24,
 <Metabolite e_Sterol__cytop at 0x1f57f286190>: -0.0014,
 <Metabolite e_DNA__cytop at 0x1f57f26e880>: -0.01028,
 <Metabolite e_Cofactor__cytop at 0x1f57f33e0a0>: -0.00103,
 <Metabolite e_Protein__cytop at 0x1f57f2f7430>: -0.1672,
 <Metabolite C00008__cytop at 0x1f57f2df700>: 39.24,
 <Metabolite e_Biomass__cytop at 0x1f57f2587f0>: 1.0,
 <Metabolite C00009__cytop at 0x1f57f2c9040>: 39.24,
 <Metabolite C00080__cytop at 0x1f57f29a430>: 39.24}

model.reactions.DM_C00369__chlo

model.reactions.e_Lipid__cytop.add_metabolites({model.metabolites.C00422__lip: 0.13})

model.reactions.e_Lipid__cytop.metabolites

model.reactions.e_Carbohydrate__cytop.metabolites

0.3212*(1-(0.0123373*48660.08/1000*0.50869+0.13*904.7821/1000*0.11407))

(1-(0.0123373*48660.08/1000*0.50869+0.13*904.7821/1000*0.11407))

model.reactions.DM_C00369__chlo.bounds = (0.3/48660.08*1000*0.21, 1000)

model.reactions.DM_C00369__chlo

model.summary()